# Re-connect to container, or throw away

We can use the computational environment in a container in many ways, including:

* Option 1: to execute one command that produces some output, and writes this to a mounted folder (so the data is available on the host file system). We don't need the computational environment any more (all our output is on the host file system). If we need to run the command again, we can start with a fresh container again.

* Option 2: to start a container as a process, to which we can connect repeatedly to carry out tasks inside the same container instance. This is known as running the container in a "detached" state.

In this notebook, we will explore both options.


## Opion 1: Run command in container, then throw container away

This is what we have done in previous examples. 

However, there is one detail that I'd like to introduce now: we executed our command, for example using ``docker run cowimage cowsay "Hello"``. Every time we run this command, docker saves the state of the container after the execution of the command, and if we really wanted, we could connect to it and carry on working with this container.

Here is some evidence. First, let's show that a new container is created every time we run the command:

In [1]:
!docker run cowimage cowsay "Hello 1"

 _________
< Hello 1 >
 ---------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


In [2]:
!docker run cowimage cowsay "Hello 2"

 _________
< Hello 2 >
 ---------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


Now we ask docker to display the directory (``ls``) of the ``last 2`` containers that have been created:

In [3]:
!docker container ls --last 2

CONTAINER ID        IMAGE               COMMAND              CREATED             STATUS                      PORTS               NAMES
d3d165e8858c        cowimage            "cowsay 'Hello 2'"   13 seconds ago      Exited (0) 12 seconds ago                       admiring_brown
416e170dc3ad        cowimage            "cowsay 'Hello 1'"   15 seconds ago      Exited (0) 13 seconds ago                       trusting_williams


We can see that there are two images, and by looking at the command and the time ago, we can see that we have just created those two containers.

We can delete those using the ``docker container rm`` command, but need to provide the ``CONTAINER ID`` as an argument. The ``CONTAINER ID`` is given in the first column of the output above. There is a switch ``-q`` to only return that:

In [4]:
!docker container ls --last 2 -q

d3d165e8858c
416e170dc3ad


With this information, we can delete those containers, for example:

In [5]:
!docker container rm $(docker container ls --last 2 -q)

d3d165e8858c
416e170dc3ad


Let's check they have disappeared:

In [6]:
!docker container ls --last 2

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


## The `run --rm` option

For one-time execution of a command, we can use the ``--rm`` option to run, which means that the container will be destroyed as soon as our use of it has stopped:

In [7]:
!docker container ls --last 1

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [8]:
!docker run --rm cowimage cowsay "Hello 3"

 _________
< Hello 3 >
 ---------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||


In [9]:
!docker container ls --last 1

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


## Option 2: attach to running container

### Start the container in detached mode using ``run -d``

To demonstrate this, we start a ``bash`` process in our container. The ``-d`` switch means to Detach the container, meaning that it will run it the background. The ``-t`` switch is necessary to attach a pseudo terminal to the ``bash`` command. The return value is the ID of the container. 

In [10]:
!docker run -d -t cowimage bash

6ba4f1e46f452b8be22ba86531b2307f3799c718ee8865149d320660cbf70924


For this demonstration, it is convenient to know the name of the container in a variable. We thus use Jupyter Notebook python magic and save the output from the ``!docker ...`` command in the variable ``output``:

In [11]:
output = !docker run -d -t cowimage bash

Turns out that ``output`` is a list:

In [12]:
output

['227518a5658159522c7f377cd45cd92782aff361d0c07a580cb513bcc660e927']

So we take the first element to be able to refer to the container ID:

In [13]:
ID = output[0]

In [14]:
ID

'227518a5658159522c7f377cd45cd92782aff361d0c07a580cb513bcc660e927'

Let's check that the container is running:

In [15]:
!docker container ls --last 1

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
227518a56581        cowimage            "bash"              7 seconds ago       Up 6 seconds                            stupefied_elbakyan


As we have stored the name of the container in the variable ``ID``, we can pass the value of the variable to the ``!docker`` command using ``$ID``. Here is an example, asking the container for the current date and time (in its timezone):

In [16]:
!docker exec $ID date

Sun Sep  9 06:55:49 UTC 2018


### Execute arbitrary commands in the running container using ``exec``

Here is the plan: using subsequent commands, we show the files in /tmp in the container, create a file in the container in /tmp, then confirm it exists, then stop the container, then start the container again, and confirm the file is still there.

Confirm there are no files in the ``/tmp/`` directory in the container:

In [17]:
!docker exec $ID ls -l /tmp


total 0


Now we create a file:

In [18]:
!docker exec $ID touch /tmp/my-file.txt

We confirm the file exists:

In [19]:
!docker exec $ID ls -l /tmp

total 0
-rw-r--r-- 1 root root 0 Sep  9 06:56 my-file.txt


As we have seen earlier, carrying out the same sequence of commands using ``run`` instead of ``exec`` would not work: as the second ``run`` command will start from the original docker *image* again and create a new container for the execution.

We can also stop our running container using ``stop``:

In [20]:
!docker stop $ID

227518a5658159522c7f377cd45cd92782aff361d0c07a580cb513bcc660e927


We cannot execute further commands in this container because it is stopped:

In [21]:
!docker exec $ID ls -l /tmp

Error response from daemon: Container 227518a5658159522c7f377cd45cd92782aff361d0c07a580cb513bcc660e927 is not running


However, we can start it again using ``start``:

In [22]:
!docker start $ID

227518a5658159522c7f377cd45cd92782aff361d0c07a580cb513bcc660e927


In [23]:
!docker exec $ID ls -l /tmp

total 0
-rw-r--r-- 1 root root 0 Sep  9 06:56 my-file.txt


## The ``container ls`` command (``ps``)

This is a good opportunity for some more detail on the ``docker container ls`` command:

Reminder: the ``ls`` command lists all *containers*. Containers are instances of images that are created when an image is being executed. 

The ``ls`` command shows all running containers:

In [24]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
227518a56581        cowimage            "bash"              37 seconds ago      Up 4 seconds                            stupefied_elbakyan
6ba4f1e46f45        cowimage            "bash"              48 seconds ago      Up 48 seconds                           compassionate_fermat


If we stop our container, it will not be displayed anymore using ``ls``:

In [25]:
!docker stop $ID

227518a5658159522c7f377cd45cd92782aff361d0c07a580cb513bcc660e927


In [26]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
6ba4f1e46f45        cowimage            "bash"              56 seconds ago      Up 55 seconds                           compassionate_fermat


### ``ls -a``

We can see all containers (even those not executing) using the ``ls -a`` option. Note that a container that is run running has the status ``Exited``, whereas the active processes show ``Up [some time]``:

In [27]:
!docker container ls -a

CONTAINER ID        IMAGE               COMMAND             CREATED              STATUS                     PORTS               NAMES
227518a56581        cowimage            "bash"              49 seconds ago       Exited (0) 4 seconds ago                       stupefied_elbakyan
6ba4f1e46f45        cowimage            "bash"              About a minute ago   Up 59 seconds                                  compassionate_fermat


### ``--last N``
The ``--last N`` option shows the last N containers that have been created (irrespective of if they are running or not). For example the last 3 containers:

In [28]:
!docker container ls --last 3

CONTAINER ID        IMAGE               COMMAND             CREATED              STATUS                     PORTS               NAMES
227518a56581        cowimage            "bash"              51 seconds ago       Exited (0) 6 seconds ago                       stupefied_elbakyan
6ba4f1e46f45        cowimage            "bash"              About a minute ago   Up About a minute                              compassionate_fermat


### ``container ls`` == ``ps``

As the ``container ls`` command behaves a little bit like the Linux ``ps`` command, we can use ``docker ps`` instead of ``docker container ls``:

In [29]:
!docker container ls --last 1

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                      PORTS               NAMES
227518a56581        cowimage            "bash"              58 seconds ago      Exited (0) 14 seconds ago                       stupefied_elbakyan


In [30]:
!docker ps --last 1

CONTAINER ID        IMAGE               COMMAND             CREATED              STATUS                      PORTS               NAMES
227518a56581        cowimage            "bash"              About a minute ago   Exited (0) 19 seconds ago                       stupefied_elbakyan


## Remove all exited containers

To remove an exited container from our host, we can use ``docker container rm`` command:

In [31]:
!docker container rm $ID

227518a5658159522c7f377cd45cd92782aff361d0c07a580cb513bcc660e927


What is the container that has been created most recently now?

In [32]:
!docker container ls -a --last 1

CONTAINER ID        IMAGE               COMMAND             CREATED              STATUS              PORTS               NAMES
6ba4f1e46f45        cowimage            "bash"              About a minute ago   Up About a minute                       compassionate_fermat


If we don't need any state that maybe saved in our exited containers, we can remove them from our host system. We can ask for the container ids (``-q``) for all containers:

In [33]:
!docker container ls -a -q


6ba4f1e46f45


and we can use that information to pass all the container ids to the ``rm`` command:

In [34]:
!docker container rm $(docker container ls -a -q)

Error response from daemon: You cannot remove a running container 6ba4f1e46f452b8be22ba86531b2307f3799c718ee8865149d320660cbf70924. Stop the container before attempting removal or force remove


In [35]:
!docker container ls -a

CONTAINER ID        IMAGE               COMMAND             CREATED              STATUS              PORTS               NAMES
6ba4f1e46f45        cowimage            "bash"              About a minute ago   Up About a minute                       compassionate_fermat


We can stop individual containers using the ``stop`` command (see above). To stop all running containers conveniently, we can use the ``kill`` command:

In [36]:
!docker container kill $(docker container ls -q)

6ba4f1e46f45


## More convenience: naming containers

In the example above, we have used the ID of the container. We can also give containers a name ourselves (ideally one that is easier to remember) and use that to refer to the container:

In [37]:
!docker run -d -t --name myname cowimage bash

04c4e348b76fc1a6cfbbe30d902241faf4eca032360f70552dc0002dadee8cc4


In [38]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
04c4e348b76f        cowimage            "bash"              2 seconds ago       Up 1 second                             myname


In [39]:
!docker exec myname date

Sun Sep  9 06:57:23 UTC 2018


In [40]:
!docker container stop myname

myname


In [41]:
!docker container rm myname

myname


It is also worth noting that we don't need to use the full hash of the container ID when referring to them; we can just use the leading few characters as long as they are unique:

In [42]:
output = !docker run -dt --name myname cowimage bash

In [43]:
IDfull = output[0]
print(IDfull)

e14eba99c7b0121fe5168e59362cbd5a6ed5786795f83f0be57a3eafb435fe8d


In [44]:
IDshort = IDfull[0:3]
print(IDshort)

e14


In [45]:
!docker container exec $IDshort date

Sun Sep  9 06:57:37 UTC 2018


In [46]:
!docker container exec $IDfull date

Sun Sep  9 06:57:37 UTC 2018


And some tidying up:

In [47]:
!docker container stop $IDshort

e14


In [48]:
!docker container rm $IDshort

e14


## More convenience: dropping ``container`` from commands

In many places where we have used ``container`` above, this can be dropped from the command:

In [49]:
!docker run -dt --name myname2 cowimage bash

88dac8aaa738d89d3e6706328076db43bdaac9d48252c0f8f54b6b7a87da2c93


In [50]:
!docker exec myname2 date

Sun Sep  9 06:57:44 UTC 2018


In [51]:
!docker stop myname2

myname2


In [52]:
!docker rm myname2

myname2


Using ``docker container exec`` and ``docker container stop`` is sligthly clearer that ``docker exec`` and ``docker stop`` and done here for education purposes, but typically people will omit the ``container`` from the commands.